In [ ]:
!pip install simpletransformers

## SQAD 

Zobaczmy jak to działa: https://rajpurkar.github.io/SQuAD-explorer/explore/v2.0/dev/

In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

In [ ]:
import json


with open('train-v2.0.json', 'r') as f:
    train_data = json.load(f)
with open('dev-v2.0.json', 'r') as f:
    val_data = json.load(f)



train_data = [item for topic in train_data['data'] for item in topic['paragraphs'] ][:100]
val_data = [item for topic in val_data['data'] for item in topic['paragraphs'] ][:50]

In [ ]:
print(len(train_data))
print(len(val_data))
print(train_data[25])

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

In [ ]:
import os

from simpletransformers.question_answering import QuestionAnsweringModel


train_args = {
    'learning_rate': 3e-5,
    'num_train_epochs': 2,
    'max_seq_length': 384,
    'doc_stride': 128,
    'overwrite_output_dir': True,
    'reprocess_input_data': False,
    'train_batch_size': 2,
    'gradient_accumulation_steps': 8, #less memory, longer time
    'evaluate_during_training': True,
    'wandb_project': "qa_bert"
}

model = QuestionAnsweringModel('bert', 'bert-base-cased', args=train_args)

# Train the model
model.train_model(train_data, eval_data=val_data)





In [ ]:
predictions, raw_outputs = model.predict(val_data)

os.makedirs('results', exist_ok=True)


submission = {pred['id']: pred['answer'] for pred in predictions}

with open('results/submission.json', 'w') as f:
    json.dump(submission, f)